# 目的

single agent, multi agent, openai assistantで同じタスクを実行してアウトプットの違いを比較したい。

- 複雑なタスクのほうが違いが出やすいと思われる。singleではプロンプトが膨らんでいき、ノイズが多いと応答の質も悪くなりそう。
- multi agentでもstateをすべて共有するか、結果だけ共有するかで結果が変わる。今回は後者の検証。
- assistantsはsingle agentと近い設計だが、FWをかますことでプロンプトに違いが出そう。そもそもassistant専用のAPIを叩いているのでchatと別物。

どんなタスクを検証するか。
- toolを使う前提で、かつtool(関数)の結果が大きい
- あとは単純だがステップ数が多いタスクで、伝言ゲームみたいに途中でデータが変わったり、目的を見失ったりしないか

方針
- まずそれぞれミニマムで組む
- tool呼び出しは共通のモックデータを返す

関数定義
- 店舗取得: 店舗IDを渡すと店舗情報（id, 店舗名, 都道府県, 会社id）を返す
- 会社取得: 会社IDを渡すと会社情報（id, 会社名, 店舗数）を返す
- 口コミ取得: 店舗IDを渡すと口コミ（店舗id, comment, review_name）一覧500件を返す
- 売り上げ取得: 店舗IDを渡すと店舗売り上げ（店舗id, 日付, 売り上げ）を直近365日分返す

## 関数定義

In [1]:
import uuid
import json
from typing import Annotated, Literal, List, Optional, Dict, Any
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command
from typing_extensions import TypedDict
from IPython.display import Image, display
from datetime import datetime, timedelta
import random
from operator import add

@tool
def get_company_ratings() -> List[Dict[str, Any]]:
    """
    Retrieve company ratings for 100 companies.

    Returns:
        List[Dict[str, Any]]: A list of company ratings with company_id and rating.
    """
    print("Function: get_company_ratings, Args: None")
    return [{"company_id": i, "rating": 5 if i == 1 else ((i % 3) + 1)} for i in range(1, 101)]

@tool
def get_stores(company_id: int) -> List[Dict[str, Any]]:
    """
    Fetch all stores operated by a specific company.

    Args:
        company_id (int): The ID of the company.

    Returns:
        List[Dict[str, Any]]: A list of store details.
    """
    print(f"Function: get_stores, Args: company_id={company_id}")
    return [
        {"store_id": 1, "name": "ラーメン一番", "prefecture": "東京都", "company_id": 1},
        {"store_id": 2, "name": "寿司太郎", "prefecture": "大阪府", "company_id": 1},
        {"store_id": 3, "name": "焼肉キング", "prefecture": "福岡県", "company_id": 1},
    ]

@tool
def get_company_info(company_id: int) -> Dict[str, Any]:
    """
    Retrieve company details based on company ID.

    Args:
        company_id (int): The ID of the company.

    Returns:
        Dict[str, Any]: Company information.
    """
    print(f"Function: get_company_info, Args: company_id={company_id}")
    if company_id == 1:
        return {"company_id": 1, "name": "グルメフーズ株式会社", "store_count": 3}
    return {}

@tool
def get_store_reviews(store_id: int) -> List[Dict[str, Any]]:
    """
    Fetch customer reviews for a specific store.

    Args:
        store_id (int): The store ID.

    Returns:
        List[Dict[str, Any]]: A list of reviews.
    """
    print(f"Function: get_store_reviews, Args: store_id={store_id}")
    comments = {
        1: [
            "接客態度が悪かった", "店員の対応が冷たい", "サービスが悪い", "注文を間違えられた", "待ち時間が長すぎる",
            "愛想がなかった", "清潔感がない", "店内がうるさい", "価格が高い割にサービスが悪い", "二度と行きたくない"
        ],
        2: [
            "料理がとても美味しかった", "味が絶品だった", "また来たいと思った", "食材が新鮮だった", "コスパが良かった",
            "雰囲気が最高だった", "シェフの腕が素晴らしい", "デザートまで完璧だった", "量もちょうど良い", "家族で楽しめた"
        ]
    }
    
    return [{"store_id": store_id, "comment": random.choice(comments.get(store_id, ["レビューなし"])), "review_name": f"user_{i}"} for i in range(1, 251)]

@tool
def get_store_sales(store_id: int) -> List[Dict[str, Any]]:
    """
    Retrieve sales data for the past 365 days for a specific store.

    Args:
        store_id (int): The store ID.

    Returns:
        List[Dict[str, Any]]: A list of daily sales data.
    """
    print(f"Function: get_store_sales, Args: store_id={store_id}")
    start_date = datetime.today() - timedelta(days=365)
    sales_data = []
    
    for i in range(365):
        date = start_date + timedelta(days=i)
        
        if store_id == 1:  # ラーメン一番 売上が急激に下がる
            sales = max(3000 - (i * 30), 100)
        elif store_id == 2:  # 寿司太郎 売上が急激に上がる
            sales = min(3000 + (i * 30), 10000)
        elif store_id == 3:  # 焼肉キング 売上は一定（1と2の中間の数値で安定）
            sales = 3000
        else:
            return []
        
        sales_data.append({"store_id": store_id, "date": date.strftime('%Y-%m-%d'), "sales": sales})
    
    return sales_data

class State(TypedDict):
    messages: Annotated[List, add_messages]

## SingleAgent

In [2]:
class SingleAgent:

    def __init__(self, model_name: str):
        """
        Initialize SingleAgent with the given model name.
        """
        self.model_name = model_name
        self.llm = ChatOpenAI(model=model_name, temperature=0)
        
        # Define tools
        self.tools = [get_company_ratings, get_stores, get_company_info, get_store_reviews, get_store_sales]
        self.llm_with_tools = self.llm.bind_tools(self.tools)
        
        # Define graph
        self.graph_builder = StateGraph(State)
        self.graph_builder.add_node("chatbot", self.chatbot)
        tool_node = ToolNode(tools=self.tools)
        self.graph_builder.add_node("tools", tool_node)
        self.graph_builder.add_conditional_edges("chatbot", tools_condition)
        self.graph_builder.add_edge("tools", "chatbot")
        self.graph_builder.add_edge(START, "chatbot")
        
        # Compile graph
        self.thread_id = uuid.uuid4()
        self.memory = MemorySaver()
        self.graph = self.graph_builder.compile(checkpointer=self.memory)
    
    def chatbot(self, state: State):
        """
        Process chatbot response.
        """
        message = self.llm_with_tools.invoke(state["messages"])
        return {"messages": [message]}
    
    def run(self, command: str) -> str:
        """
        Execute the chatbot and return the response as a string.
        """
        config = {"configurable": {"thread_id": self.thread_id}}
            
        events = self.graph.invoke(
            {"messages": [{"role": "user", "content": command}]},
            config,
            stream_mode="values",
        )
        return events['messages'][-1].content

In [3]:
single_agent = SingleAgent("gpt-4o-mini")
result = single_agent.run("こんにちは、あなたの機能を教えてください。")
print(result)

こんにちは！私はさまざまな情報を提供するための機能を持っています。具体的には、以下のことができます：

1. **企業の評価を取得**: 100社の企業の評価を取得できます。
2. **店舗情報の取得**: 特定の企業が運営するすべての店舗の情報を取得できます。
3. **企業の詳細情報の取得**: 企業IDに基づいて企業の詳細情報を取得できます。
4. **店舗の顧客レビューの取得**: 特定の店舗に対する顧客のレビューを取得できます。
5. **店舗の売上データの取得**: 特定の店舗の過去365日間の売上データを取得できます。

何か特定の情報が必要でしたら、お知らせください！


In [4]:
result = single_agent.run("""
次の指示に従い、企業の口コミを分析して結果をレポートにまとめてください。
1. 全企業で最も評価が高い企業を分析の対象とします
2. 対象の売上を調べ、売上が好調な店舗と不調な店舗をそれぞれ1店舗ずつ抽出します
3. 抽出した店舗の口コミを調べ、売上好調の理由と売上不調の理由をそれぞれ分析します
""")
print(result)

# 企業名と店舗名を間違える
# 店舗数も間違える
# 売上が伸びてる店舗を間違える

Function: get_company_ratings, Args: None
Function: get_stores, Args: company_id=1
Function: get_store_sales, Args: store_id=1
Function: get_store_sales, Args: store_id=2
Function: get_store_reviews, Args: store_id=1
Function: get_store_reviews, Args: store_id=2
### 企業の口コミ分析レポート

#### 1. 分析対象企業
最も評価が高い企業は「ラーメン一番」で、評価は5です。

#### 2. 売上データ
- **売上好調な店舗**: 
  - 店舗名: ラーメン一番
  - 売上データ: 売上は3000円から始まり、徐々に減少していく傾向が見られますが、初期の売上は高いです。

- **売上不調な店舗**: 
  - 店舗名: 寿司太郎
  - 売上データ: 売上は3000円から始まり、徐々に増加し、最終的には10000円に達しています。

#### 3. 口コミ分析
- **ラーメン一番（売上好調）**
  - 口コミの内容は主にネガティブで、以下のようなコメントが多く見られました。
    - 「サービスが悪い」
    - 「清潔感がない」
    - 「店員の対応が冷たい」
  - **分析**: 売上は高いものの、顧客満足度は低いことが示唆されます。高い売上は、立地やブランド力によるもので、顧客のリピート率が低い可能性があります。

- **寿司太郎（売上不調）**
  - 口コミの内容はポジティブで、以下のようなコメントが多く見られました。
    - 「料理がとても美味しい」
    - 「新鮮な食材を使用している」
    - 「家族で楽しめる」
  - **分析**: 売上は徐々に増加しているものの、初期の売上が低いのは、認知度やマーケティング不足が原因かもしれません。顧客の満足度は高いが、集客が課題です。

### 結論
- **ラーメン一番**は高評価にもかかわらず、顧客満足度が低く、リピート率が低い可能性があります。改善点としては、サービスの向上や店舗の清掃が挙げられます。
-

## MultiAgent

In [19]:
class ReviewAgent:
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.llm = ChatOpenAI(model=model_name, temperature=0)
        
        # Define tools
        self.tools = [get_store_reviews]
        self.llm_with_tools = self.llm.bind_tools(self.tools)
        
        # Define graph
        self.graph_builder = StateGraph(State)
        self.graph_builder.add_node("chatbot", self.chatbot)
        tool_node = ToolNode(tools=self.tools)
        self.graph_builder.add_node("tools", tool_node)
        self.graph_builder.add_conditional_edges("chatbot", tools_condition)
        self.graph_builder.add_edge("tools", "chatbot")
        self.graph_builder.add_edge(START, "chatbot")
        
        # Compile graph
        self.thread_id = uuid.uuid4()
        self.memory = MemorySaver()
        self.graph = self.graph_builder.compile(checkpointer=self.memory)
    
    def chatbot(self, state: State):
        """
        Process chatbot response.
        """
        message = self.llm_with_tools.invoke(state["messages"])
        return {"messages": [message]}
    
    def run(self, command: str) -> str:
        """
        Execute the chatbot and return the response as a string.
        """
        config = {"configurable": {"thread_id": self.thread_id}}
            
        events = self.graph.invoke(
            {"messages": [{"role": "user", "content": command}]},
            config,
            stream_mode="values",
        )
        return events['messages'][-1].content

class SaleAgent:
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.llm = ChatOpenAI(model=model_name, temperature=0)
        
        # Define tools
        self.tools = [get_store_sales]
        self.llm_with_tools = self.llm.bind_tools(self.tools)
        
        # Define graph
        self.graph_builder = StateGraph(State)
        self.graph_builder.add_node("chatbot", self.chatbot)
        tool_node = ToolNode(tools=self.tools)
        self.graph_builder.add_node("tools", tool_node)
        self.graph_builder.add_conditional_edges("chatbot", tools_condition)
        self.graph_builder.add_edge("tools", "chatbot")
        self.graph_builder.add_edge(START, "chatbot")
        
        # Compile graph
        self.thread_id = uuid.uuid4()
        self.memory = MemorySaver()
        self.graph = self.graph_builder.compile(checkpointer=self.memory)
    
    def chatbot(self, state: State):
        """
        Process chatbot response.
        """
        message = self.llm_with_tools.invoke(state["messages"])
        return {"messages": [message]}
    
    def run(self, command: str) -> str:
        """
        Execute the chatbot and return the response as a string.
        """
        config = {"configurable": {"thread_id": self.thread_id}}
            
        events = self.graph.invoke(
            {"messages": [{"role": "user", "content": command}]},
            config,
            stream_mode="values",
        )
        return events['messages'][-1].content

class CompanyAgent:
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.llm = ChatOpenAI(model=model_name, temperature=0)
        
        # Define tools
        self.tools = [get_company_ratings, get_stores, get_company_info]
        self.llm_with_tools = self.llm.bind_tools(self.tools)
        
        # Define graph
        self.graph_builder = StateGraph(State)
        self.graph_builder.add_node("chatbot", self.chatbot)
        tool_node = ToolNode(tools=self.tools)
        self.graph_builder.add_node("tools", tool_node)
        self.graph_builder.add_conditional_edges("chatbot", tools_condition)
        self.graph_builder.add_edge("tools", "chatbot")
        self.graph_builder.add_edge(START, "chatbot")
        
        # Compile graph
        self.thread_id = uuid.uuid4()
        self.memory = MemorySaver()
        self.graph = self.graph_builder.compile(checkpointer=self.memory)
    
    def chatbot(self, state: State):
        """
        Process chatbot response.
        """
        message = self.llm_with_tools.invoke(state["messages"])
        return {"messages": [message]}
    
    def run(self, command: str) -> str:
        """
        Execute the chatbot and return the response as a string.
        """
        config = {"configurable": {"thread_id": self.thread_id}}
            
        events = self.graph.invoke(
            {"messages": [{"role": "user", "content": command}]},
            config,
            stream_mode="values",
        )
        return events['messages'][-1].content
    

@tool
def analysis_review(
    store_id: int,
    query: str
) -> str:
    """
    Retrieves store reviews based on the specified store_id and query, returning the analyzed results.

    Args:
        store_id (int): The store ID.
        query (str): The specific aspect of reviews to analyze.

    Returns:
        str: Store review analysis data.
    """
    try:
        print(f"Function: analysis_review, Args: store_id={store_id}, query={query}")
        review_agent = ReviewAgent("gpt-4o-mini")
        messages = [
            {
                "role": "user",
                "content": f"""
                以下の条件で口コミを分析しなさい。
                店舗ID: {store_id}
                知りたいこと: {query}
                """
            }
        ]
        response = review_agent.run(messages[-1]["content"])
        return response
    except Exception as e:
        print(f"Error in analysis_review: {e}")
        raise

@tool
def analysis_sales(
    store_id: int,
    query: str
) -> str:
    """
    Retrieves sales data for a given store ID and provides an analysis based on the specified query.
    
    Args:
        store_id (int): The store ID for which sales analysis is required.
        query (str): The specific aspect of sales data to analyze.

    Returns:
        str: A sales performance analysis including key insights and recommendations.
    """
    print(f"Function: analysis_sales, Args: store_id={store_id}, query={query}")
    sale_agent = SaleAgent("gpt-4o-mini")
    messages = [
        {
            "role": "user",
            "content": f"""
            以下の条件で売上を分析しなさい。
            店舗ID: {store_id}
            知りたいこと: {query}
            """
        }
    ]
    response = sale_agent.run(messages[-1]["content"])
    return response

@tool
def analysis_company(
    company_id: int,
    query: str
) -> str:
    """
    Retrieves company-level data, including store performance, company ratings, and other relevant metrics, based on the specified query.
    
    Args:
        company_id (int): The company ID for which analysis is required.
        query (str): The specific aspect of company performance to analyze.

    Returns:
        str: A comprehensive analysis of the company's performance, including insights on the specified query.
    """
    print(f"Function: analysis_company, Args: company_id={company_id}, query={query}")
    company_agent = CompanyAgent("gpt-4o-mini")
    messages = [
        {
            "role": "user",
            "content": f"""
            以下の条件で売上を分析しなさい。
            企業ID: {company_id}
            知りたいこと: {query}
            """
        }
    ]
    response = company_agent.run(messages[-1]["content"])
    return response


class MultiAgent:
    def __init__(self, model_name: str):
        """
        Initialize SingleAgent with the given model name.
        """
        self.model_name = model_name
        self.llm = ChatOpenAI(model=model_name, temperature=0)
        
        # Define tools
        self.tools = [analysis_review, analysis_sales, analysis_company]
        self.llm_with_tools = self.llm.bind_tools(self.tools)
        
        # Define graph
        self.graph_builder = StateGraph(State)
        self.graph_builder.add_node("chatbot", self.chatbot)
        tool_node = ToolNode(tools=self.tools)
        self.graph_builder.add_node("tools", tool_node)
        self.graph_builder.add_conditional_edges("chatbot", tools_condition)
        self.graph_builder.add_edge("tools", "chatbot")
        self.graph_builder.add_edge(START, "chatbot")
        
        # Compile graph
        self.thread_id = uuid.uuid4()
        self.memory = MemorySaver()
        self.graph = self.graph_builder.compile(checkpointer=self.memory)
    
    def chatbot(self, state: State):
        """
        Process chatbot response.
        """
        message = self.llm_with_tools.invoke(state["messages"])
        return {"messages": [message]}
    
    def run(self, command: str) -> str:
        """
        Execute the chatbot and return the response as a string.
        """
        config = {"configurable": {"thread_id": self.thread_id}}
            
        events = self.graph.invoke(
            {"messages": [{"role": "user", "content": command}]},
            config,
            stream_mode="values",
        )
        return events['messages'][-1].content

In [6]:
agent = ReviewAgent("gpt-4o-mini")
result = agent.run("店舗ID2の口コミを分析して改善案を教えてください")
print(result)

Function: get_store_reviews, Args: store_id=2
店舗ID2の口コミを分析した結果、以下のポイントが浮かび上がりました。

### ポジティブなフィードバック
1. **家族向けの楽しさ**: 多くの口コミで「家族で楽しめた」というコメントがあり、家族連れに人気のある店舗であることがわかります。
2. **雰囲気の良さ**: 「雰囲気が最高だった」という意見が多数あり、居心地の良い空間が提供されていることが評価されています。
3. **料理の質**: 「味が絶品だった」「料理がとても美味しかった」というコメントが多く、料理のクオリティが高いことが示されています。
4. **コストパフォーマンス**: 「コスパが良かった」という意見も多く、価格に対する満足度が高いことが伺えます。
5. **デザートの評価**: 「デザートまで完璧だった」というコメントがあり、デザートの質も高いと評価されています。
6. **シェフの腕**: 「シェフの腕が素晴らしい」という意見が多く、料理の技術が高いことが強調されています。
7. **新鮮な食材**: 「食材が新鮮だった」というコメントもあり、食材の質に対する評価が高いです。

### 改善案
1. **メニューの多様性**: 家族連れに人気があるため、子供向けのメニューやアレルギー対応メニューを増やすことで、さらに幅広い顧客層を取り込むことができるかもしれません。
2. **特別イベントの開催**: 家族向けの特別イベント（例：子供向けの料理教室やファミリーデー）を開催することで、リピーターを増やすことが期待できます。
3. **口コミの活用**: 口コミでの評価が高い点を強調したマーケティングを行い、SNSや広告でのプロモーションに活用することが効果的です。
4. **顧客フィードバックの収集**: 定期的に顧客からのフィードバックを収集し、改善点を見つけることで、サービスの向上につなげることができます。
5. **季節限定メニューの導入**: 季節ごとの特別メニューを導入することで、リピーターを促進し、常に新しい体験を提供することができます。

これらの改善案を実施することで、さらに顧客満足度を向上させ、リピーターを増やすことができるでしょう。


In [7]:
agent = SaleAgent("gpt-4o-mini")
result = agent.run("店舗ID2の売上を分析して結果をまとめてください")
print(result)

Function: get_store_sales, Args: store_id=2
店舗ID2の売上データを分析した結果を以下にまとめます。

### 売上データ概要
- **期間**: 2024年2月1日から2025年1月30日までの365日間
- **売上の変動**: 売上は日々増加しており、特に2024年2月から2024年10月にかけては、毎日一定の増加が見られました。
- **最高売上**: 2024年9月22日から2025年1月30日までの期間は、売上が毎日10,000円で安定しています。

### 売上のトレンド
- **初期の成長**: 2024年2月の初めには、売上は3,000円から始まり、毎日30円ずつ増加していきました。
- **成長の加速**: 2024年3月から6月にかけて、売上は徐々に増加し、最終的には7,500円に達しました。
- **安定期**: 2024年9月22日以降、売上は10,000円で安定し、変動が見られませんでした。

### 売上の数値
- **最初の売上**: 3,000円（2024年2月1日）
- **最終売上**: 10,000円（2024年9月22日以降）

### 結論
店舗ID2は、初期の段階で売上が着実に増加し、その後安定した売上を維持しています。この安定した売上は、店舗の運営やマーケティング戦略が成功していることを示唆しています。今後の戦略としては、売上のさらなる増加を目指すための新しい施策やプロモーションを検討することが重要です。


In [8]:
agent = CompanyAgent("gpt-4o-mini")
result = agent.run("最も評価が高い企業と、その企業が運営する店舗の情報を調査して報告してください。")
print(result)

Function: get_company_ratings, Args: None
Function: get_company_info, Args: company_id=1
Function: get_stores, Args: company_id=1
最も評価が高い企業は「グルメフーズ株式会社」で、評価は5です。この企業が運営する店舗の情報は以下の通りです。

### グルメフーズ株式会社
- **店舗数**: 3

#### 店舗情報
1. **ラーメン一番**
   - **所在地**: 東京都
2. **寿司太郎**
   - **所在地**: 大阪府
3. **焼肉キング**
   - **所在地**: 福岡県

この企業は、ラーメン、寿司、焼肉の店舗を展開しており、各地で人気を集めています。


In [9]:
multi_agent = MultiAgent("gpt-4o-mini")
result = multi_agent.run("こんにちは、あなたの機能を教えてください。")
print(result)

こんにちは！私はさまざまなデータ分析を行うことができます。具体的には、以下のような機能があります：

1. **店舗レビュー分析**: 特定の店舗のレビューを分析し、顧客の意見や感想を把握することができます。

2. **売上データ分析**: 特定の店舗の売上データを分析し、パフォーマンスやトレンドに関する洞察を提供します。

3. **企業パフォーマンス分析**: 企業全体のパフォーマンスを分析し、店舗のパフォーマンスや企業の評価に関する情報を提供します。

これらの機能を使って、特定の店舗や企業に関する詳細な情報を得ることができます。何か特定のリクエストがありますか？


In [10]:
result = multi_agent.run("""
次の指示に従い、企業の口コミを分析して結果をレポートにまとめてください。
1. 全企業で最も評価が高い企業を分析の対象とします
2. 対象の売上を調べ、売上が好調な店舗と不調な店舗をそれぞれ1店舗ずつ抽出します
3. 抽出した店舗の口コミを調べ、売上好調の理由と売上不調の理由をそれぞれ分析します
""")
print(result)

Function: analysis_company, Args: company_id=1, query=highest rated company
Function: get_company_ratings, Args: None
Function: get_company_info, Args: company_id=1
Function: get_company_info, Args: company_id=2
Function: get_company_info, Args: company_id=3
Function: get_company_info, Args: company_id=4
Function: get_company_info, Args: company_id=5
Function: get_company_info, Args: company_id=6
Function: get_company_info, Args: company_id=7
Function: get_company_info, Args: company_id=8
Function: get_company_info, Args: company_id=9
Function: get_company_info, Args: company_id=10
Function: get_company_info, Args: company_id=11
Function: get_company_info, Args: company_id=12
Function: get_company_info, Args: company_id=13
Function: get_company_info, Args: company_id=14
Function: get_company_info, Args: company_id=15
Function: get_company_info, Args: company_id=16
Function: get_company_info, Args: company_id=17
Function: get_company_info, Args: company_id=18
Function: get_company_info,

In [20]:
multi_agent = MultiAgent("gpt-4o")
result = multi_agent.run("""
次の指示に従い、企業の口コミを分析して結果をレポートにまとめてください。
1. 全企業で最も評価が高い企業を分析の対象とします
2. 対象の売上を調べ、売上が好調な店舗と不調な店舗をそれぞれ1店舗ずつ抽出します
3. 抽出した店舗の口コミを調べ、売上好調の理由と売上不調の理由をそれぞれ分析します
""")
print(result)

Function: analysis_company, Args: company_id=1, query=全企業で最も評価が高い企業を特定する
Function: get_company_ratings, Args: None
Function: analysis_sales, Args: store_id=1, query=売上が好調な店舗と不調な店舗をそれぞれ1店舗ずつ抽出する
Function: get_store_sales, Args: store_id=1
Function: analysis_review, Args: store_id=1, query=売上が好調な理由を分析する
Function: analysis_review, Args: store_id=1, query=売上が不調な理由を分析する
Function: get_store_reviews, Args: store_id=1
Function: get_store_reviews, Args: store_id=1
以下は、全企業で最も評価が高い企業（企業ID: 1）の店舗に関する分析結果です。

### 売上が好調な店舗の分析
- **売上が好調な理由**:
  - **立地の良さ**: 店舗が人通りの多い場所に位置している可能性があり、集客力が高い。
  - **リピーターの存在**: 一部の顧客がリピーターである場合、悪い体験があっても再訪することがある。
  - **プロモーションやキャンペーン**: 特別なプロモーションや割引が行われている場合、顧客を引き寄せる要因となる。
  - **競合との差別化**: 競合他社に比べて独自の魅力や商品がある場合、顧客が集まることがある。

### 売上が不調な店舗の分析
- **売上が不調な理由**:
  - **接客態度の悪さ**: 多くの口コミで「接客態度が悪かった」「愛想がなかった」「店員の対応が冷たい」といったネガティブなコメントが見受けられ、これにより、顧客が不快な思いをし、再訪を避ける原因となっている。
  - **注文ミス**: 「注文を間違えられた」というコメントが多数あり、顧客の期待を裏切る結果となっている。
  - **待ち時間の長さ**: 「待ち時間が長すぎる」という意見が多く、顧客がサービスを受けるまで

## OpenAI Assistant

In [11]:
def get_company_ratings() -> List[Dict[str, Any]]:
    """
    Retrieve company ratings for 100 companies.

    Returns:
        List[Dict[str, Any]]: A list of company ratings with company_id and rating.
    """
    print("Function: get_company_ratings, Args: None")
    return [{"company_id": i, "rating": 5 if i == 1 else ((i % 3) + 1)} for i in range(1, 101)]

def get_stores(company_id: int) -> List[Dict[str, Any]]:
    """
    Fetch all stores operated by a specific company.

    Args:
        company_id (int): The ID of the company.

    Returns:
        List[Dict[str, Any]]: A list of store details.
    """
    print(f"Function: get_stores, Args: company_id={company_id}")
    return [
        {"store_id": 1, "name": "ラーメン一番", "prefecture": "東京都", "company_id": 1},
        {"store_id": 2, "name": "寿司太郎", "prefecture": "大阪府", "company_id": 1},
        {"store_id": 3, "name": "焼肉キング", "prefecture": "福岡県", "company_id": 1},
    ]

def get_company_info(company_id: int) -> Dict[str, Any]:
    """
    Retrieve company details based on company ID.

    Args:
        company_id (int): The ID of the company.

    Returns:
        Dict[str, Any]: Company information.
    """
    print(f"Function: get_company_info, Args: company_id={company_id}")
    if company_id == 1:
        return {"company_id": 1, "name": "グルメフーズ株式会社", "store_count": 3}
    return {}

def get_store_reviews(store_id: int) -> List[Dict[str, Any]]:
    """
    Fetch customer reviews for a specific store.

    Args:
        store_id (int): The store ID.

    Returns:
        List[Dict[str, Any]]: A list of reviews.
    """
    print(f"Function: get_store_reviews, Args: store_id={store_id}")
    comments = {
        1: [
            "接客態度が悪かった", "店員の対応が冷たい", "サービスが悪い", "注文を間違えられた", "待ち時間が長すぎる",
            "愛想がなかった", "清潔感がない", "店内がうるさい", "価格が高い割にサービスが悪い", "二度と行きたくない"
        ],
        2: [
            "料理がとても美味しかった", "味が絶品だった", "また来たいと思った", "食材が新鮮だった", "コスパが良かった",
            "雰囲気が最高だった", "シェフの腕が素晴らしい", "デザートまで完璧だった", "量もちょうど良い", "家族で楽しめた"
        ]
    }
    
    return [{"store_id": store_id, "comment": random.choice(comments.get(store_id, ["レビューなし"])), "review_name": f"user_{i}"} for i in range(1, 251)]

def get_store_sales(store_id: int) -> List[Dict[str, Any]]:
    """
    Retrieve sales data for the past 365 days for a specific store.

    Args:
        store_id (int): The store ID.

    Returns:
        List[Dict[str, Any]]: A list of daily sales data.
    """
    print(f"Function: get_store_sales, Args: store_id={store_id}")
    start_date = datetime.today() - timedelta(days=365)
    sales_data = []
    
    for i in range(365):
        date = start_date + timedelta(days=i)
        if store_id == 1:  # ラーメン一番 売上が急激に下がる
            sales = max(3000 - (i * 30), 100)
        elif store_id == 2:  # 寿司太郎 売上が急激に上がる
            sales = min(3000 + (i * 30), 10000)
        elif store_id == 3:  # 焼肉キング 売上は一定（1と2の中間の数値で安定）
            sales = 3000
        else:
            return []
        
        sales_data.append({"store_id": store_id, "date": date.strftime('%Y-%m-%d'), "sales": sales})
    
    return sales_data

In [12]:
from openai import OpenAI
import openai

# Define the functions
functions = [
    {
        "type": "function",
        "function": {
            "name": "get_company_ratings",
            "description": "Retrieve company ratings for 100 companies.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_stores",
            "description": "Fetch all stores operated by a specific company.",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_id": {
                        "type": "integer",
                        "description": "The ID of the company."
                    }
                },
                "required": ["company_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_company_info",
            "description": "Retrieve company details based on company ID.",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_id": {
                        "type": "integer",
                        "description": "The ID of the company."
                    }
                },
                "required": ["company_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_store_reviews",
            "description": "Fetch customer reviews for a specific store.",
            "parameters": {
                "type": "object",
                "properties": {
                    "store_id": {
                        "type": "integer",
                        "description": "The store ID."
                    }
                },
                "required": ["store_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_store_sales",
            "description": "Retrieve sales data for the past 365 days for a specific store.",
            "parameters": {
                "type": "object",
                "properties": {
                    "store_id": {
                        "type": "integer",
                        "description": "The store ID."
                    }
                },
                "required": ["store_id"]
            }
        }
    }
]

client = openai.Client()

# 実行のたび作られるのでコメントアウト
# # Create the assistant
# assistant_4omini = client.beta.assistants.create(
#     instructions="You are an assistant that provides information about companies and their stores. Use the provided functions to answer questions.",
#     model="gpt-4o-mini",
#     tools=functions
# )

# # Create the assistant
# assistant_4o = client.beta.assistants.create(
#     instructions="You are an assistant that provides information about companies and their stores. Use the provided functions to answer questions.",
#     model="gpt-4o",
#     tools=functions
# )

In [13]:
# print("Assistant ID gpt-4o-mini:", assistant_4omini.id)
# Assistant ID gpt-4o-mini: asst_1HJ2pLSKLpeCByKHRpFUG2Fs

# print("Assistant ID      gpt-4o:", assistant_4o.id)
# Assistant ID      gpt-4o: asst_wLK8W6YTxKLv62i2PawJmxZY

In [14]:
import openai


class Assistant:
    def __init__(self, model: str):
        self.client = openai.OpenAI()  # OpenAIクライアントのインスタンス
        self.assistant_id = {
            "gpt-4o-mini": "asst_1HJ2pLSKLpeCByKHRpFUG2Fs",
            "gpt-4o": "asst_wLK8W6YTxKLv62i2PawJmxZY",
        }.get(model)

        if not self.assistant_id:
            raise ValueError("Invalid model name. Choose 'gpt-4o-mini' or 'gpt-4o'.")

        # スレッドの作成（初期化時に一度だけ生成）
        self.thread = self.client.beta.threads.create()

    def run(self, command: str) -> str:
        # メッセージをスレッドに追加
        self.client.beta.threads.messages.create(
            thread_id=self.thread.id,
            role="user",
            content=command,
        )

        # Assistantの実行
        run = self.client.beta.threads.runs.create_and_poll(
            thread_id=self.thread.id,
            assistant_id=self.assistant_id,
        )

        # 必要なアクション（ツール実行）の処理
        while run.status == "requires_action":
            tool_outputs = []
            for tool_call in run.required_action.submit_tool_outputs.tool_calls:
                tool_outputs.append(self.handle_tool_call(tool_call))

            if tool_outputs:
                run = self.client.beta.threads.runs.submit_tool_outputs_and_poll(
                    thread_id=self.thread.id,
                    run_id=run.id,
                    tool_outputs=tool_outputs,
                )

        # 結果を取得
        if run.status == "completed":
            messages = self.client.beta.threads.messages.list(thread_id=self.thread.id)

            # assistant のレスポンスを抽出（複数のコンテンツがある場合も対応）
            responses = []
            for msg in messages.data:
                if msg.role == "assistant":
                    for content in msg.content:
                        if content.type == "text":
                            responses.append(content.text.value)

            return "\n".join(responses) if responses else "No response from assistant."
        else:
            return f"Error: {run.status}"

    def handle_tool_call(self, tool_call):
        """ツール呼び出しを処理し、適切なレスポンスを返す"""
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        if function_name == "get_company_ratings":
            output = self.get_company_ratings()
        elif function_name == "get_stores":
            output = self.get_stores(arguments["company_id"])
        elif function_name == "get_company_info":
            output = self.get_company_info(arguments["company_id"])
        elif function_name == "get_store_reviews":
            output = self.get_store_reviews(arguments["store_id"])
        elif function_name == "get_store_sales":
            output = self.get_store_sales(arguments["store_id"])
        else:
            output = "Unknown function"

        return {"tool_call_id": tool_call.id, "output": json.dumps(output, ensure_ascii=False)}

    def get_company_ratings(self) -> List[Dict[str, Any]]:
        """
        Retrieve company ratings for 100 companies.

        Returns:
            List[Dict[str, Any]]: A list of company ratings with company_id and rating.
        """
        print("Function: get_company_ratings, Args: None")
        return [
            {"company_id": i, "rating": 5 if i == 1 else ((i % 3) + 1)}
            for i in range(1, 101)
        ]

    def get_stores(self, company_id: int) -> List[Dict[str, Any]]:
        """
        Fetch all stores operated by a specific company.

        Args:
            company_id (int): The ID of the company.

        Returns:
            List[Dict[str, Any]]: A list of store details.
        """
        print(f"Function: get_stores, Args: company_id={company_id}")
        return [
            {
                "store_id": 1,
                "name": "ラーメン一番",
                "prefecture": "東京都",
                "company_id": 1,
            },
            {
                "store_id": 2,
                "name": "寿司太郎",
                "prefecture": "大阪府",
                "company_id": 1,
            },
            {
                "store_id": 3,
                "name": "焼肉キング",
                "prefecture": "福岡県",
                "company_id": 1,
            },
        ]

    def get_company_info(self, company_id: int) -> Dict[str, Any]:
        """
        Retrieve company details based on company ID.

        Args:
            company_id (int): The ID of the company.

        Returns:
            Dict[str, Any]: Company information.
        """
        print(f"Function: get_company_info, Args: company_id={company_id}")
        if company_id == 1:
            return {"company_id": 1, "name": "グルメフーズ株式会社", "store_count": 3}
        return {}

    def get_store_reviews(self, store_id: int) -> List[Dict[str, Any]]:
        """
        Fetch customer reviews for a specific store.

        Args:
            store_id (int): The store ID.

        Returns:
            List[Dict[str, Any]]: A list of reviews.
        """
        print(f"Function: get_store_reviews, Args: store_id={store_id}")
        comments = {
            1: [
                "接客態度が悪かった",
                "店員の対応が冷たい",
                "サービスが悪い",
                "注文を間違えられた",
                "待ち時間が長すぎる",
                "愛想がなかった",
                "清潔感がない",
                "店内がうるさい",
                "価格が高い割にサービスが悪い",
                "二度と行きたくない",
            ],
            2: [
                "料理がとても美味しかった",
                "味が絶品だった",
                "また来たいと思った",
                "食材が新鮮だった",
                "コスパが良かった",
                "雰囲気が最高だった",
                "シェフの腕が素晴らしい",
                "デザートまで完璧だった",
                "量もちょうど良い",
                "家族で楽しめた",
            ],
        }

        return [
            {
                "store_id": store_id,
                "comment": random.choice(comments.get(store_id, ["レビューなし"])),
                "review_name": f"user_{i}",
            }
            for i in range(1, 251)
        ]

    def get_store_sales(self, store_id: int) -> List[Dict[str, Any]]:
        """
        Retrieve sales data for the past 365 days for a specific store.

        Args:
            store_id (int): The store ID.

        Returns:
            List[Dict[str, Any]]: A list of daily sales data.
        """
        print(f"Function: get_store_sales, Args: store_id={store_id}")
        start_date = datetime.today() - timedelta(days=365)
        sales_data = []

        for i in range(365):
            date = start_date + timedelta(days=i)
            if store_id == 1:  # ラーメン一番 売上が急激に下がる
                sales = max(3000 - (i * 30), 100)
            elif store_id == 2:  # 寿司太郎 売上が急激に上がる
                sales = min(3000 + (i * 30), 10000)
            elif store_id == 3:  # 焼肉キング 売上は一定（1と2の中間の数値で安定）
                sales = 3000
            else:
                return []

            sales_data.append(
                {
                    "store_id": store_id,
                    "date": date.strftime("%Y-%m-%d"),
                    "sales": sales,
                }
            )

        return sales_data


# 使用例
# assistant = Assistant("gpt-4o")
# response = assistant.run("こんにちは、あなたの機能を教えてください。")
# print(response)


In [15]:
assistant = Assistant("gpt-4o-mini")
result = assistant.run("こんにちは、あなたの機能を教えてください。")
print(result)

こんにちは！私は会社やその店舗に関する情報を提供するためのアシスタントです。具体的には、以下のことができます：

1. **企業の評価を取得**：企業の評価や評判に関する情報を提供します。
2. **店舗の情報を取得**：特定の企業が運営する店舗のリストを取得します。
3. **企業の詳細情報を取得**：企業の基本情報や業務内容を提供します。
4. **店舗の顧客レビューを取得**：特定の店舗に関する顧客のレビューを取得します。
5. **店舗の売上データを取得**：特定の店舗の過去365日間の売上データを取得します。

何か特定の情報が必要でしたら、お知らせください！


In [16]:
assistant = Assistant("gpt-4o-mini")
result = assistant.run("""
次の指示に従い、企業の口コミを分析して結果をレポートにまとめてください。
1. 全企業で最も評価が高い企業を分析の対象とします
2. 対象の売上を調べ、売上が好調な店舗と不調な店舗をそれぞれ1店舗ずつ抽出します
3. 抽出した店舗の口コミを調べ、売上好調の理由と売上不調の理由をそれぞれ分析します
""")
print(result)

Function: get_company_ratings, Args: None
Function: get_stores, Args: company_id=1
Function: get_store_sales, Args: store_id=1
Function: get_store_sales, Args: store_id=2
Function: get_store_sales, Args: store_id=3
Function: get_store_reviews, Args: store_id=1
Function: get_store_reviews, Args: store_id=3
## 企業評価分析レポート

### 1. 企業選定
全企業の中で最も評価が高い企業は「ラーメン一番」で、評価は5です。

### 2. 売上データ
#### 店舗: ラーメン一番

- **売上好調店舗**: ^寿司太郎^（大阪府）
  - 売上傾向: 売上は着実に増加し、最終的には10,000を超える。
  
- **売上不調店舗**: ^焼肉キング^（福岡県）
  - 売上傾向: 売上は初期は3000から始まり、その後は一定期間横ばいで推移。

### 3. 口コミ分析
#### 売上好調店舗: 寿司太郎
- **主な口コミ**:
  - 評価が高く、顧客の満足度が非常に良い。
- **ポジティブな理由**:
  - 新鮮な食材を使用している。
  - スタッフのサービスが良い。
  - レストランの清掃状態が良好。

#### 売上不調店舗: 焼肉キング
- **主な口コミ**:
  - 再訪したいという意見は少ない。
  - スタッフの対応が悪いという投稿が多数。
- **ネガティブな理由**:
  - 店内が清潔でないとの指摘が多数。
  - 注文する際のミスが多い。
  - 値段に見合った食材の質が悪いといったクレーム。

## 結論
- 売上好調の店舗「寿司太郎」は、顧客サービスや品質に対する評価が高く、これが売上の向上につながっています。一方、売上不調の「焼肉キング」は、清掃の行き届いた店舗管理や顧客サービスの改善が求められます。これらの点を踏まえ、各店舗は今後の戦略を見直す必要があります。
